In [2]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
import gmaps

# Import the API key.
from config import weather_api_key
from config import g_key

# Import the datetime module from the datetime library.
from datetime import datetime

# Import time module
import time

# Import linregress
from scipy.stats import linregress

In [3]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)

lat_lngs = zip(lats, lngs) 

In [4]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

737

In [6]:
# Starting URL for Weather Map API Call.
weather_url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [14]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities): # same thing as for i in range(len(cities)):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = weather_url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_wtype = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_wtype,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 2 of Set 1 | poronaysk
Processing Record 3 of Set 1 | kyren
Processing Record 4 of Set 1 | rikitea
Processing Record 5 of Set 1 | ushuaia
Processing Record 6 of Set 1 | mount gambier
Processing Record 7 of Set 1 | new norfolk
Processing Record 8 of Set 1 | castro
Processing Record 9 of Set 1 | poum
Processing Record 10 of Set 1 | mustafakemalpasa
Processing Record 11 of Set 1 | hami
Processing Record 12 of Set 1 | sojat
Processing Record 13 of Set 1 | koygorodok
Processing Record 14 of Set 1 | labuhan
Processing Record 15 of Set 1 | mar del plata
Processing Record 16 of Set 1 | albany
Processing Record 17 of Set 1 | palana
Processing Record 18 of Set 1 | cape town
Processing Record 19 of Set 1 | punta arenas
Processing Record 20 of Set 1 | port lincoln
Processing Record 21 of Set 1 | chokurdakh
Processing Record 22 of Set 1 | kavaratti
Pro

In [16]:
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Poronaysk,RU,2021-08-07 02:48:22,49.2167,143.1167,59.00,88,100,8.50,overcast clouds
1,Kyren,RU,2021-08-07 02:44:04,51.6828,102.1408,63.03,75,100,1.45,overcast clouds
2,Rikitea,PF,2021-08-07 02:48:22,-23.1203,-134.9692,74.77,76,9,10.54,clear sky
3,Ushuaia,AR,2021-08-07 02:48:23,-54.8000,-68.3000,33.46,80,75,21.85,light snow
4,Mount Gambier,AU,2021-08-07 02:44:05,-37.8333,140.7667,51.71,87,75,17.27,broken clouds
5,New Norfolk,AU,2021-08-07 02:44:05,-42.7826,147.0587,51.66,66,80,5.01,broken clouds
6,Castro,BR,2021-08-07 02:44:05,-24.7911,-50.0119,53.44,98,99,5.12,overcast clouds
7,Poum,NC,2021-08-07 02:44:06,-20.2333,164.0167,68.97,64,100,6.42,overcast clouds
8,Mustafakemalpasa,TR,2021-08-07 02:48:23,40.0353,28.4117,77.05,83,66,2.51,broken clouds
9,Hami,CN,2021-08-07 02:44:07,42.8000,93.4500,83.16,16,100,5.12,overcast clouds


In [17]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")